In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import time

import sys
sys.path.append('..')
from input_pipeline import Pipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/mnt/datasets/dan/moda/edanet/train/shard-0000.tfrecords',
    '/mnt/datasets/dan/moda/edanet/train/shard-0001.tfrecords',
    '/mnt/datasets/dan/moda/edanet/train/shard-0002.tfrecords',
]
params = {
    'batch_size': 16, 'image_height': 256, 'image_width': 256,
}

with tf.device('/cpu:0'):
    pipeline = Pipeline(files, is_training=True, params=params)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()

# Show an image

In [ ]:
color_by_label = {j: np.random.randint(0, 255, size=3, dtype='uint8') for j in range(13)}

def get_color_mask(sparse_mask):
    masks = []
    for j in range(13):
        m = (sparse_mask == (j + 1)).astype('uint8')
        masks.append(np.expand_dims(m, 2) * color_by_label[j])
        
    return np.stack(masks).max(0)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output_images, output_masks = sess.run([features, labels])

In [ ]:
i = 9
image = (255.0*output_images[i]).astype('uint8')
Image.fromarray(image)

# Show masks

In [ ]:
t = Image.fromarray(image)
t.putalpha(255)

mask = get_color_mask(output_masks[i])
m = Image.fromarray(mask)
m.putalpha(Image.fromarray(150*(mask > 0).any(2).astype('uint8')))
t.alpha_composite(m)
t

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run([features, labels])
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())